#Import Kaggle Dataset

##Import Kaggle Api

In [ ]:
!gdown 19-HJSodf1BC2e5bTVIBUgrdhWxtO5vMx
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=19-HJSodf1BC2e5bTVIBUgrdhWxtO5vMx
To: /content/kaggle.json
100% 62.0/62.0 [00:00<00:00, 316kB/s]


##Import Dataset

In [ ]:
# Import datasets
# !kaggle datasets download -d promptcloud/amazon-product-dataset-2020
# !unzip 'amazon-product-dataset-2020.zip'
# !mv './home/sdf/marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv' '[01] Kaggle - Amazon Product.csv'
# !rm -r './home' 'amazon-product-dataset-2020.zip'

!kaggle datasets download -d mervemenekse/ecommerce-dataset
!unzip 'ecommerce-dataset.zip'
!mv 'E-commerce Dataset.csv' '[02] E-commerce.csv'
!rm -r  'ecommerce-dataset.zip'

!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store
!unzip 'ecommerce-behavior-data-from-multi-category-store.zip'
!mv '2019-Oct.csv' '[03] E-commerce Multi-cat - part1.csv'
# !mv '2019-Nov.csv' '[03] E-commerce Multi-cat - part2.csv'
!rm -r  '2019-Nov.csv' 'ecommerce-behavior-data-from-multi-category-store.zip'

!kaggle datasets download -d harishedison/kpmg-customer-demography-cleaned-dataset
!unzip 'kpmg-customer-demography-cleaned-dataset.zip'
!mv 'KPMG_Customer_Demography.csv' '[04] Customer Demog.csv'
!rm -r  'kpmg-customer-demography-cleaned-dataset.zip'

# !kaggle datasets download -d thedevastator/demographical-shopping-purchases-data
# !unzip 'demographical-shopping-purchases-data.zip'
# !mv 'Demographic_Data_Orig.csv' '[05] Customer Demog2.csv'
# !rm -r  'demographical-shopping-purchases-data.zip'

!kaggle datasets download -d kartik2112/fraud-detection
!unzip 'fraud-detection.zip'
!mv 'fraudTest.csv' '[06] Fraud Vendor.csv'
!rm 'fraud-detection.zip' 'fraudTrain.csv'

##Clean up Kaggle API

In [ ]:
!rm -r ~/.kaggle

#General LIB Import

In [ ]:
import pandas as pd;
import numpy as np;
import re;
import random;
import hashlib;

#Generating Vendor

In [ ]:
class VendorDBGenerator:
    RAND_SEED = 487590;
    SRC_06 = '[06] Fraud Vendor.csv';
    REF_INIT_DT = pd.Timestamp('2022-11-01 00:00:00');

    fraud_df: pd.DataFrame = None;
    vendor_df: pd. DataFrame = None;

    def __init__(self):
        np.random.seed(self.RAND_SEED);
        pass;

    def get_fraud(self) -> pd.DataFrame:
        if self.fraud_df is None:
            df:pd.DataFrame = pd.read_csv(self.SRC_06)
            self.fraud_df = df;
        return self.fraud_df[:];

    def set_src(self, src:pd.DataFrame): self.fraud_df = src;

    def extract_vendor(self) -> pd.DataFrame:
        if self.vendor_df is None:
            df:pd.DataFrame = self.get_fraud()[['merchant', 'cc_num', 'street', 'state', 'zip']];

            df['biz_alias'] = df['merchant'].str.replace(pat='fraud_', repl='');
            df.drop_duplicates( subset= ['biz_alias'], inplace=True, ignore_index=True);
            df['prev_biz_alias'] = None;

            df['id'] = df.index+1;

            df['biz_account'] = df['biz_alias'].str.lower();
            df['biz_account'] = df['biz_account'].apply(lambda x: re.findall('^[a-z]+', x)[0]);
            df['biz_account'] = df[['id', 'biz_account']].apply(lambda x: '{}_{}'.format(x['biz_account'], x['id']), axis='columns');

            df['id'] = df['id'].astype(str);
            df['id'] = 'VEND'+df['id'].str.zfill(4);

            df['email'] = df['biz_account'].apply(lambda x: x+'@vend.com');
            df['country'] = 'Australia';
            df['prev_addr'] = None;

            t_df: pd.DataFrame = self.gen_phone()[3801:];
            t_df.reset_index(drop=True, inplace=True);
            df = df.join(t_df);

            df = df.join(self.gen_official_flag(df.size));
            df = df.join(self.gen_reg_dt(df.size));
            df['updated_dt'] = df['registered_dt'];

            df['zip'] = df['zip'].astype(str);
            df['zip_code'] = df['zip'].apply( lambda x: x[-4:]);
            df['zip_code'] = df['zip_code'].apply( lambda x: '2'+x[1:] if x[0]=='0' else x);
            df['state'] = df['zip_code'].apply(lambda x: VendorDBGenerator.zip2state(x));

            df.rename(columns = {'street': 'street_addr'}, inplace=True);

            self.vendor_df = df[[ 'id', 'biz_account', 'email', 'biz_alias', 'prev_biz_alias', 'phone',
                'street_addr', 'state', 'zip_code',
                'country', 'prev_addr',
                'official_flag', 'registered_dt', 'updated_dt']];
        return self.vendor_df;

    def gen_phone(self):
        t_no = self.get_fraud()[['cc_num']].drop_duplicates();
        t_no['cc_num'] = t_no['cc_num'].astype(str);
        t1 = t_no['cc_num'].apply(lambda x: x[:9]);
        t2 = t_no['cc_num'].apply(lambda x: x[1:10]);
        t3 = t_no['cc_num'].apply(lambda x: x[2:11]);
        t4 = t_no['cc_num'].apply(lambda x: x[3:12]);
        t5 = t_no['cc_num'].apply(lambda x: x[4:13]);
        s: pd.Series = pd.concat([t1, t2, t3, t4, t5], ignore_index=True).drop_duplicates();
        s.name = 'phone';
        s = '0'+s;
        s = s.str.ljust(width=10, fillchar='0');
        return s.to_frame();

    def gen_official_flag(self, size:int) -> pd.DataFrame:
        data = np.random.normal(0,1,size=size);
        df: pd.DataFrame = pd.DataFrame(data=data, dtype='float16', columns=['score'])
        df['official_flag'] = ~(df['score'].between(-1,1));
        return df[['official_flag']];

    def gen_reg_dt(self, size:int) -> pd.DataFrame:
        data = np.random.gamma(2,36,size=size);
        df:pd.DataFrame = pd.DataFrame(data=data, dtype='float32', columns=['delta']);
        df.sort_values(by=['delta'], ignore_index=True, inplace=True);
        df['delta'] = df['delta'].apply(lambda x: pd.Timedelta(round(x*864000), unit='sec'));
        df['registered_dt'] = self.REF_INIT_DT + df['delta'];
        return df[['registered_dt']];

    def zip2state(zip_code:str) -> str:
        z:str = zip_code[0];
        if (z == '1') | (z=='2') : return 'New South Wales';
        if (z == '3') | (z=='8') : return 'Victoria';
        if (z == '4') | (z=='9') : return 'Queensland';
        if z == '5' : return 'South Australia';
        if z == '6' : return 'Western Australia';
        if z == '7' : return 'Tasmania';
        return None;

#Generating Customer

In [ ]:
class CustDBGenerator:
    SRC_02 = '[02] E-commerce.csv';
    SRC_04 = '[04] Customer Demog.csv';
    SRC_05 = '[05] Customer Demog2.csv';
    SRC_06 = '[06] Fraud Vendor.csv';

    REF_CURRENT_DT = pd.Timestamp('2023-04-15 23:59:59');
    RAND_SEED = 487590;

    fraud_df: pd.DataFrame = None;
    ecomm_df: pd.DataFrame = None;
    demog_df: pd.DataFrame = None;

    tgt_df: pd.DataFrame = None;

    def __init__(self):
        random.seed(self.RAND_SEED);
        pass;

    def get_fraud(self) -> pd.DataFrame:
        if self.fraud_df is None:
            self.fraud_df = pd.read_csv(self.SRC_06);
        return self.fraud_df[:];

    def get_ecomm(self) -> pd.DataFrame:
        if self.ecomm_df is None:
            self.ecomm_df = pd.read_csv(self.SRC_02);
        return self.ecomm_df[:];

    def get_demog(self) -> pd.DataFrame:
        if self.demog_df is None:
            self.demog_df = pd.read_csv(self.SRC_04);
        return self.demog_df[:];

    def set_src(self, mock:pd.DataFrame, ecom:pd.DataFrame, demog:pd.DataFrame):
        self.fraud_df = mock;
        self.ecomm_df = ecom;
        self.demog_df = demog;
        pass;

    def gen_phone(self):
        t_no = self.get_fraud()[['cc_num']].drop_duplicates();
        t_no['cc_num'] = t_no['cc_num'].astype(str);
        t1 = t_no['cc_num'].apply(lambda x: x[:9]);
        t2 = t_no['cc_num'].apply(lambda x: x[1:10]);
        t3 = t_no['cc_num'].apply(lambda x: x[2:11]);
        t4 = t_no['cc_num'].apply(lambda x: x[3:12]);
        t5 = t_no['cc_num'].apply(lambda x: x[4:13]);
        s: pd.Series = pd.concat([t1, t2, t3, t4, t5], ignore_index=True).drop_duplicates();
        s.name = 'phone';
        s = '0'+s;
        s = s.str.ljust(width=10, fillchar='0');
        return s.to_frame();

    def gen_datetime(self) -> pd.DataFrame:
        df:pd.DataFrame = self.get_ecomm()[['Customer_Id','Aging']];
        df.dropna(subset=['Aging'], inplace=True);
        df.columns = ['ref_id', 'ref_age'];
        df['id'] = df['ref_id'] % 3000 +1;
        df.drop_duplicates(subset=['id'], inplace=True);
        df['tdelta'] = df['ref_age'].apply(lambda x: pd.Timedelta(round((10*x+random.random()*10)*86400), unit='sec'));
        df['registered_dt'] = self.REF_CURRENT_DT;
        df['registered_dt'] = df['registered_dt'] - df['tdelta'];
        df['updated_dt'] = df['registered_dt'];
        df.sort_values(by='registered_dt', inplace=True);
        df.reset_index(drop=True, inplace=True);
        return df[['registered_dt', 'updated_dt']][:];

    def gen_demog(self) -> pd.DataFrame:
        df:pd.DataFrame = self.get_demog()[['customer_id', 'First_Name', 'Last_Name', 'gender', 'DOB',
                    'Address', 'Postcode', 'State', 'Country']];
        df.columns = df.columns.str.lower();

        df['dob'] = pd.to_datetime(df['dob'], format='%m/%d/%Y')
        df['inval_age_flag'] = pd.to_numeric(df['dob'].dt.strftime('%Y')) < 1967
        df.drop( df[ df['inval_age_flag'] == True ].index, inplace = True);

        df['name'] = df[['first_name', 'last_name']].apply(lambda x: '{} {}'.format(x['first_name'], x['last_name']), axis='columns');

        df.reset_index(drop=True, inplace=True)
        df['id'] = df.index + 1;
        df['skey_id'] = df['id'];
        df.drop(df[df['id']>3000].index, inplace=True);

        df['email'] = df[['id', 'last_name']].apply(lambda x: '{}_{}@mail.com'.format(x['last_name'], x['id']), axis='columns');
        df['alt_email'] = None;
        df['validated_flag']=True;

        df['id'] = df['id'].astype(str);
        df['id'] = 'CID'+df['id'].str.zfill(5);

        t_df: pd.DataFrame = self.gen_phone()[:3000];
        t_df.reset_index(drop=True, inplace=True);
        df = df.join(t_df);

        df.rename(columns={'postcode':'zip_code', 'address': 'street_addr'}, inplace=True);
        return df[['id','skey_id', 'email', 'phone', 'name', 'gender', 'dob', 'street_addr', 'zip_code', 'state', 'country', 'alt_email', 'validated_flag']][:];

    def extract_customer(self) -> pd.DataFrame:
        if self.tgt_df is None:
            df: pd.DataFrame = self.gen_demog();
            df_x: pd.DataFrame = self.gen_datetime();
            df = df.join(df_x);
            self.tgt_df = df[['id','skey_id', 'email', 'phone', 'name', 'gender', 'dob',
                    'street_addr', 'zip_code', 'state', 'country', 'alt_email',
                    'registered_dt', 'updated_dt', 'validated_flag']];
        return self.tgt_df;

#Generate Product and Category DB

In [ ]:
class ProdxTxDBGenerator:
    RAND_SEED = 487590;
    REF_INIT_DT = pd.Timestamp('2022-11-01 00:00:00');

    PROD_FIRST_REGIST_DT = pd.Timestamp('2022-11-15 00:01:01');
    # PROD_LAST_REGIST_DT = pd.Timestamp('2022-12-31 00:01:01');

    TRX_EST_FIRST_TS = pd.Timestamp('2023-01-08 00:00:01');
    TRX_EST_LAST_TS = pd.Timestamp('2023-04-11 00:00:01');

    SRC_03_1:str = '[03] E-commerce Multi-cat - part1.csv';

    src_trx_df: pd.DataFrame = None;
    src_vend_df: pd.DataFrame = None;
    src_cust_df: pd.DataFrame = None;

    trx_df: pd.DataFrame = None;
    cat_df: pd.DataFrame = None;

    prodx_df: pd.DataFrame = None;
    pid_map: pd.DataFrame = None;

    def get_src_trx(self) -> pd.DataFrame:
        if self.src_trx_df is None:
            icols = ['event_time', 'product_id','category_code', 'brand', 'price', 'user_id'];
            df: pd.DataFrame = pd.read_csv(self.SRC_03_1, nrows=200000)[icols];
            self.src_trx_df = df;
        return self.src_trx_df.loc[:]

    def get_src_vendor(self) -> pd.DataFrame:
        if self.src_vend_df is None:
            self.src_vend_df = VendorDBGenerator().extract_vendor();
        return self.src_vend_df.loc[:];
    def get_src_customer(self) -> pd.DataFrame:
        if self.src_cust_df is None:
            self.src_cust_df = CustDBGenerator().extract_customer();
        return self.src_cust_df.loc[:];

    def set_srcs(self, vendor:pd.DataFrame, customer:pd.DataFrame, trx:pd.DataFrame=None):
        self.src_trx_df = self.get_src_trx() if trx is None else trx;
        self.src_vend_df = vendor;
        self.src_cust_df = customer;
        return self;

    def extract_prod_cat(self) -> pd.DataFrame:
        if self.cat_df is None:
            df: pd.DataFrame = self.get_src_trx().dropna(subset=['product_id', 'category_code', 'brand', 'price'])[['category_code']];
            df.drop_duplicates(inplace=True, ignore_index=True);
            df.rename(columns={'category_code': 'code'}, inplace=True);
            df.sort_values(by='code', inplace=True, ignore_index=True);

            df['id'] = df.index+1;
            df['id'] = df['id'].astype(str);
            df['id'] = 'CAT' +df['id'].str.zfill(4);

            df['label'] =  df['code'].str.split('.');
            df['label'] = df['label'].apply(lambda x: x[-1]).str.replace('_', ' ').str.title();

            df['desc'] = df['code'].str.replace(r'\.|_', ' ', regex=True).str.capitalize();

            df['created_dt'] = self.REF_INIT_DT;
            df['updated_dt'] = self.REF_INIT_DT;
            df['deactivated_flag'] = False;
            self.cat_df = df[[ 'id', 'code', 'label', 'desc', 'created_dt', 'updated_dt', 'deactivated_flag']];
        return self.cat_df.loc[:];

    def get_prod_cat_map(self) -> pd.DataFrame: return self.extract_prod_cat()[['code', 'id']].rename(columns={'id':'prod_cat_id'});

    def report_prd_cat_issue(self):
        cat:pd.DataFrame = self.extract_prod_cat();
        cat['raw'] = cat['cat_code'].str.split('.');
        cat['len'] = cat['raw'].apply(lambda x: len(x));
        w:pd.Series = None;
        for i in range(0,cat.len.max()):
            tmp = cat[['raw','len']].apply(lambda x: x.raw[i] if i< x.len else None, axis='columns');
            tmp.dropna(inplace=True);
            w = tmp if w is None else pd.concat([w,tmp], ignore_index=True);
            w.drop_duplicates(inplace=True);
        w.reset_index(drop=True, inplace=True);
        label = pd.DataFrame(w,columns=['id']);
        label['label'] = label['id'];
        label.to_csv('cat_id.csv', index=False);

        link:pd.DataFrame = None
        for i in range(0, cat.len.max()-1):
            tmp = cat[['raw','len']].apply(lambda x: [x.raw[i], x.raw[i+1]] if i<x.len-1 else None, axis='columns');
            tmp.dropna(inplace=True);
            tdf = pd.DataFrame.from_dict(dict(zip(tmp.index, tmp.values)), orient='index', columns=['source', 'target']);
            link = tdf if link is None else pd.concat([link, tdf]);
            link.drop_duplicates(inplace=True);
        link.reset_index(drop=True, inplace=True);
        link.to_csv('cat_link.csv', index=False);
        pass;

    def init_prodx(self) -> pd.DataFrame:
        if self.prodx_df is None:
            df: pd.DataFrame = self.get_src_trx()[['product_id', 'category_code', 'brand', 'price']];
            df.dropna(inplace=True);
            df.drop_duplicates(inplace=True, ignore_index=True);
            df.rename(columns = {'product_id': 'id', 'category_code': 'cat_code'}, inplace=True);

            # Set price and previous price
            p_list = df.groupby(by=['id', 'cat_code', 'brand'])['price'].apply(list);
            df = df.set_index(keys=['id', 'cat_code', 'brand']).join(p_list, rsuffix='_list');
            df['plist_len'] = df['price_list'].apply(len)
            df.reset_index(inplace=True);
            df.drop_duplicates(subset=['id'], inplace=True, ignore_index=True)
            df['price'] = df['price_list'].apply(lambda x: x[-1]);
            df['prev_price'] = df[['price_list', 'plist_len']].apply(lambda x: x['price_list'][-2] if x['plist_len']>1 else None, axis='columns');
            df.drop(columns=['price_list', 'plist_len'], inplace=True);

            df['vendor_id'] = None;
            df['name'] = df['cat_code'].str.split('.');
            df['name'] = df[['name', 'brand']].apply(lambda x: '{} {}'.format(x.brand, x['name'][-1]), axis='columns');
            df['name'] = df.name.str.replace('\.|_',' ', regex=True).str.title();

            df['desc'] = df['name'] +': ' + df.cat_code.str.replace('\.|_',' ', regex=True);
            df['features'] = df.brand.apply(lambda x: {'brand': x});
            df['search_kw'] = df['brand'] + ' ' + df['cat_code'].str.replace('\.|_',' ', regex=True);
            df['search_kw'] = df['search_kw'].str.split(' ');
            df['created_dt'] = None;
            df['updated_dt'] = None;
            df['deactivated_flag'] = False;

            df['product_id'] = df['id'];
            min_pid = df['id'].min();
            df['id'] = min_pid+df.index;
            self.pid_map:pd.DataFrame = df[['product_id', 'id']].loc[:];
            self.pid_map.rename(columns={'id':'prod_id'}, inplace=True);
            self.pid_map.drop_duplicates(inplace=True, ignore_index=True);
            df.drop(columns=['product_id'], inplace=True);

            # Modify created_dt
            np.random.seed(self.RAND_SEED)
            data = np.random.normal(453, 114, size=len(df.index)-1);
            data = np.concatenate((np.array([0]), data));
            data = np.around(data);
            data = np.cumsum(data, dtype=int);
            df['delta'] = data;
            df['delta'] = df['delta'].round();
            df['delta'] = df['delta']+df['delta'].shift(1);
            df['delta'].fillna(0, inplace=True);
            df['delta'] = df['delta'].apply(lambda x: pd.Timedelta(x, unit='sec'));
            df['created_dt'] = self.PROD_FIRST_REGIST_DT + df['delta'];
            df.drop(columns=['delta'], inplace=True);

            df['updated_dt'] = df['created_dt'];

            # Map vendor id
            src_vend_df:pd.DataFrame = self.get_src_vendor();
            if src_vend_df is not None:
                vend_df:pd.DataFrame = src_vend_df[['id', 'registered_dt']].loc[:];
                df['max_vindx'] = df['created_dt'].apply(lambda x: vend_df[vend_df.registered_dt < x].index.max());
                np.random.seed(self.RAND_SEED);
                df['vindx'] = df['max_vindx'].apply(lambda x: np.random.randint(x));
                df.set_index('vindx', inplace=True);
                df = df.join(vend_df[['id']],rsuffix='_vend');
                df.reset_index(drop=True, inplace=True);
                df['vendor_id'] = df['id_vend'];
                df.drop(columns=['max_vindx', 'id_vend'], inplace=True);

            # Map category id
            cat_map:pd.DataFrame = self.get_prod_cat_map();
            cat_map.set_index('code', inplace=True);
            df.set_index('cat_code', inplace=True);
            df = df.join(cat_map);
            df.sort_values(by='id', inplace=True, ignore_index=True);

            self.prodx_df = df[['id', 'vendor_id', 'prod_cat_id',
                    'name', 'brand', 'desc', 'price',
                    'features', 'search_kw', 'prev_price',
                    'created_dt', 'updated_dt',
                    'deactivated_flag']];
        return self.prodx_df.loc[:];

    def get_pid_map(self) -> pd.DataFrame:
        if self.pid_map is None:
            self.init_prodx();
        return self.pid_map.loc[:];

    def extract_prodx(self)-> pd.DataFrame:
        prodx_df: pd.DataFrame = self.init_prodx();
        cap_trx_df:pd.DataFrame = self.extract_trx();

        # Capture price change
        cap_trx_df = cap_trx_df[['timestamp', 'prod_id', 'price']].drop_duplicates(subset=['prod_id', 'price']);
        cap_trx_df['f'] = 1;
        trx_n:pd.DataFrame = cap_trx_df.groupby(by='prod_id')['f'].sum();
        ts_list:pd.DataFrame = cap_trx_df.groupby(by='prod_id')['timestamp'].apply(list);
        price_list:pd.DataFrame = cap_trx_df.groupby(by='prod_id')['price'].apply(list);
        cap_trx_df.drop_duplicates(subset=['prod_id'], inplace=True);
        cap_trx_df.set_index('prod_id',inplace=True);
        cap_trx_df.drop(columns=['timestamp', 'price','f'], inplace=True);
        cap_trx_df = cap_trx_df.join(trx_n);
        cap_trx_df = cap_trx_df.join(ts_list);
        cap_trx_df = cap_trx_df.join(price_list);
        cap_trx_df.drop(cap_trx_df[cap_trx_df.f==1].index, inplace=True);
        cap_trx_df['trx_prev_timestamp'] = cap_trx_df['timestamp'].apply(lambda x:x[0]);
        cap_trx_df['trx_timestamp'] = cap_trx_df['timestamp'].apply(lambda x:x[1]);
        cap_trx_df['trx_price'] = cap_trx_df['price'].apply(lambda x:x[1]);
        cap_trx_df['trx_prev_price'] = cap_trx_df['price'].apply(lambda x:x[0]);
        cap_trx_df.drop(columns=['f', 'timestamp', 'price'], inplace=True);

        # Generate product update datetime
        prodx_df.set_index('id', inplace=True);
        prodx_df = prodx_df.join(cap_trx_df);
        prodx_df.reset_index(inplace=True);
        prodx_df['deltasec_ts'] = (prodx_df['trx_timestamp'] - prodx_df['trx_prev_timestamp']).dt.total_seconds();
        np.random.seed(self.RAND_SEED)
        prodx_df['r'] = pd.Series(np.random.random(len(prodx_df.index)));
        prodx_df['modified_dt'] = round(prodx_df['deltasec_ts']*prodx_df['r']);
        prodx_df['modified_dt'] = prodx_df['modified_dt'].apply(lambda x: pd.Timedelta(x, unit='sec'));
        prodx_df['modified_dt'] = prodx_df['modified_dt'] + prodx_df['trx_prev_timestamp'];
        prodx_df['updated_dt'] = prodx_df[['updated_dt', 'modified_dt']].apply(lambda x: x.updated_dt if pd.isnull(x.modified_dt) else x.modified_dt, axis='columns');
        prodx_df.drop(columns=['trx_prev_timestamp', 'trx_timestamp', 'trx_price', 'trx_prev_price', 'deltasec_ts', 'r', 'modified_dt'], inplace=True);
        return prodx_df;

    def extract_trx(self) -> pd.DataFrame:
        if self.trx_df is None:
            df: pd.DataFrame = self.get_src_trx();
            df = df.dropna(subset=['product_id', 'category_code', 'brand', 'price'])[['event_time', 'product_id', 'user_id', 'price']].loc[:];

            # Manipulate time
            df['seq'] = df.index;
            df['event_time'] = df['event_time'].str.replace('UTC','').str.strip();
            df['event_time'] = df['event_time'].apply(pd.Timestamp);

            # Map product id
            pid_map:pd.DataFrame = self.get_pid_map();
            pid_map.set_index('product_id', inplace=True);
            df.set_index('product_id', inplace=True);
            df = df.join(pid_map);
            df.reset_index(drop=True, inplace=True);

            # Retrieve product info
            prdx_df:pd.DataFrame = self.init_prodx();
            prdx_df = prdx_df[['id','created_dt']];
            prdx_df.rename(columns={'created_dt':'prd_created_dt'}, inplace=True);
            prdx_df.set_index('id', inplace=True);
            df.set_index('prod_id', inplace=True);
            df = df.join(prdx_df);
            df.reset_index(inplace=True, names=['prod_id']);
            df.sort_values(by='seq', inplace=True);

            # Generate transaction timestamp
            np.random.seed(self.RAND_SEED);
            evtime_ts:pd.DataFrame = df[['event_time']].drop_duplicates();
            evtime_min = evtime_ts['event_time'].min();
            evtime_range = evtime_ts['event_time'].max() - evtime_ts['event_time'].min();
            evtime_ts['dsec'] = (evtime_ts['event_time'] - evtime_min).dt.total_seconds();

            tscale_range = self.TRX_EST_LAST_TS - self.TRX_EST_FIRST_TS;
            tscale_ratio = round(tscale_range.total_seconds()/evtime_range.total_seconds());
            evtime_ts['scaled_dsec'] = evtime_ts['dsec'] * tscale_ratio;
            evtime_ts['scaled_dsec'] = evtime_ts['scaled_dsec'].apply(lambda x: pd.Timedelta(x, unit='sec'));

            evtime_ts['timestamp'] = self.TRX_EST_FIRST_TS + evtime_ts['scaled_dsec'];
            evtime_ts = evtime_ts.set_index('event_time')[['timestamp']];
            df = df.set_index('event_time').join(evtime_ts);
            df = df.reset_index(drop=True);
            inv_tx:pd.DataFrame = df[['seq', 'timestamp', 'prd_created_dt']].loc[:];
            inv_tx['prd_available_dt'] = inv_tx['prd_created_dt'] + pd.Timedelta(1, unit='day');
            inv_tx = inv_tx[inv_tx.timestamp < inv_tx.prd_available_dt];
            inv_tx['r'] = np.random.randint(648000, size=len(inv_tx.index));
            inv_tx['r'] = inv_tx['r'].apply( lambda x: pd.Timedelta(x,unit='sec'));
            inv_tx['timestamp'] = inv_tx['prd_available_dt'] + inv_tx['r'];
            inv_tx.set_index('seq');
            df.set_index('seq')
            df = df.join(inv_tx[['timestamp']], rsuffix='_fixed');
            df['timestamp_fixed'] = df['timestamp_fixed'].fillna(df['timestamp']);
            df.drop(columns=['seq', 'prd_created_dt', 'timestamp'], inplace=True);
            df.rename(columns={'timestamp_fixed': 'timestamp'}, inplace=True);
            df.sort_values(by='timestamp', ignore_index=True, inplace=True);

            # Generate transaction header
            np.random.seed(self.RAND_SEED);
            txid = df[['timestamp']].drop_duplicates();
            tx_id_init = np.random.randint(self.RAND_SEED);
            txid['trx_header_id'] = txid.index + tx_id_init;
            txid.set_index('timestamp', inplace=True);
            df.set_index('timestamp', inplace=True);
            df = df.join(txid);
            df.reset_index(inplace=True, names=['timestamp']);

            # Generate customer
            cust_df: pd.DataFrame = self.get_src_customer();
            cust_df = cust_df[['id','registered_dt']];
            cust_df['reg_gap_dt'] = cust_df['registered_dt'] + pd.Timedelta(1, unit='day');

            ## Get max index of customer id
            trx_ts:pd.DataFrame = df[['timestamp']].drop_duplicates();
            trx_ts['cindx_max'] = trx_ts['timestamp'].apply(lambda x: cust_df[cust_df.reg_gap_dt < x].index.max());
            trx_ts.set_index('timestamp',inplace=True);
            df = df.set_index('timestamp').join(trx_ts).reset_index();

            ## Randomly get index of customer id
            np.random.seed(self.RAND_SEED)
            trx_cindx = df[['user_id', 'trx_header_id', 'cindx_max']].drop_duplicates();
            trx_cindx['uindx'] = trx_cindx['user_id']%3000;
            trx_cindx = trx_cindx[['trx_header_id', 'uindx', 'cindx_max']].drop_duplicates();

            ## Trx with list of valid customer
            lst:pd.DataFrame = trx_cindx[trx_cindx.uindx <= trx_cindx.cindx_max].groupby('trx_header_id')['uindx'].apply(list).reset_index();
            lst['len'] = lst['uindx'].apply(len)-1;
            lst.set_index('trx_header_id', inplace=True);
            lst['r'] = lst['len'].apply(lambda x: np.random.randint(0,x) if x>0 else 0);
            lst['cindex'] = lst[['uindx', 'r']].apply(lambda x: x['uindx'][x.r], axis='columns');

            trx_cindx = trx_cindx[['trx_header_id', 'cindx_max']].drop_duplicates();
            trx_cindx.set_index('trx_header_id', inplace=True);
            trx_cindx = trx_cindx.join(lst[['cindex']]);

            ## Trx with invalid customer
            n_lst: pd.DataFrame = trx_cindx[trx_cindx.cindex.isna()][['cindx_max']];
            n_lst['cindex'] = n_lst['cindx_max'].apply(np.random.randint);

            trx_cindx = trx_cindx.join(n_lst[['cindex']], rsuffix='_r');
            trx_cindx['cindex'] = trx_cindx['cindex'].fillna(trx_cindx['cindex_r']);

            trx_cindx = trx_cindx[['cindex']].reset_index().set_index(['cindex']);
            trx_cindx = trx_cindx.join(cust_df[['id']]);
            trx_cindx.rename(columns={'id':'cust_id'}, inplace=True);
            trx_cindx.set_index('trx_header_id', inplace=True);

            df.set_index('trx_header_id', inplace=True);
            df = df.join(trx_cindx);
            df.drop(columns=['user_id', 'cindx_max'], inplace=True);
            df.reset_index(inplace=True);

            self.trx_df = df[['trx_header_id', 'cust_id', 'prod_id', 'price', 'timestamp']];
            self.trx_df = df
        return self.trx_df.loc[:];

#Generate TRX Header and TRX Item DB

In [ ]:
class TrxDbGenerator:
    RAND_SEED = 487590;
    src_tx_df:pd.DataFrame = None;
    tx_header_df: pd.DataFrame = None;
    tx_item_df: pd.DataFrame = None;

    def __init__(self, trx_df:pd.DataFrame):
        self.src_tx_df = trx_df;
        pass;

    def get_src_trx(self) -> pd.DataFrame: return self.src_tx_df.loc[:];

    def extract_trx_header(self):
        if self.tx_header_df is None:
            df:pd.DataFrame = self.get_src_trx();
            df.rename(columns={'trx_header_id':'id'}, inplace=True);
            trx_amnt:pd.DataFrame = df.groupby(by='id')[['price']].sum();
            trx_amnt.rename(columns={'price':'sales_amnt'}, inplace=True);
            df.drop_duplicates(subset=['id'], inplace=True);
            df.set_index('id', inplace=True);
            df = df.join(trx_amnt);
            df.reset_index(inplace=True);
            df.drop(columns=['prod_id', 'price'], inplace=True);
            df['cart_id'] = None;
            df['total_discount'] = 0;
            df['total_amnt'] = df['sales_amnt'] - df['total_discount'];
            self.tx_header_df = df[['id', 'cust_id', 'timestamp', 'cart_id', 'sales_amnt', 'total_discount', 'total_amnt']];
        return self.tx_header_df.loc[:];

    def extract_trx_item(self):
        if self.tx_item_df is None:
            df:pd.DataFrame = self.get_src_trx();
            df['qty'] = 1;
            trx_item = df.groupby(by=['trx_header_id', 'prod_id'])[['price', 'qty']].sum();
            trx_item.rename(columns={'price':'amnt', 'qty':'quantity'}, inplace=True);

            df.drop_duplicates(subset=['trx_header_id', 'prod_id', 'price'], inplace =True);
            df.drop(columns=['qty'], inplace =True);
            df = df.set_index(['trx_header_id', 'prod_id']).join(trx_item);
            df.reset_index(inplace=True);

            np.random.seed(self.RAND_SEED);
            indx0 = round(np.random.randint(self.RAND_SEED)*np.random.random());
            df['id'] = indx0 + df.index;
            df['hrs_incart'] = None;

            self.tx_item_df = df[['id', 'trx_header_id', 'prod_id', 'quantity', 'price', 'amnt', 'hrs_incart']];
        return self.tx_item_df.loc[:];

#Promotion DB

In [ ]:
class PromoDBGenerator:
    '''Assume promotion first lunch on 15 Mar 2023'''
    RAND_SEED = 487590;
    REF_DT_PROMO_LUNCH = pd.Timestamp('2023-03-15');

    promo_src:pd.DataFrame = None;
    trx_header_df = None;
    trx_item_df = None;

    promo_df:pd.DataFrame = None;
    prod_promo_df:pd.DataFrame = None;
    promo_redeem_df:pd.DataFrame = None;
    trx_promo_df:pd.DataFrame = None;

    final_promo_redeem_df:pd.DataFrame = None;
    final_trx_header_df:pd.DataFrame = None;


    def __init__(self, vendor:pd.DataFrame, prodx:pd.DataFrame, trx:pd.DataFrame, trx_header:pd.DataFrame):
        vendor:pd.DataFrame = vendor[['id', 'biz_alias', 'official_flag']];
        vendor.set_index('id', inplace=True);

        prodx:pd.DataFrame = prodx[['id', 'vendor_id', 'name', 'created_dt','updated_dt']];
        prodx.set_index('id', inplace=True);

        self.trx_header_df = trx_header.loc[:];
        ts_id = trx_header[(trx_header.timestamp > self.REF_DT_PROMO_LUNCH)].id;
        trx_header:pd.DataFrame = trx_header[trx_header.id.isin(ts_id)][['id', 'timestamp']];
        trx_header.set_index('id', inplace=True);

        self.trx_item_df = trx.loc[:];
        df:pd.DataFrame = trx[trx.trx_header_id.isin(ts_id)];
        df = df[['id', 'trx_header_id', 'prod_id', 'quantity', 'price']];

        df.set_index('trx_header_id', inplace=True);
        df = df.join(trx_header);
        df.reset_index(names=['trx_header_id'], inplace=True);

        df.set_index('prod_id', inplace=True);
        df = df.join(prodx);
        df.reset_index(names=['prod_id'], inplace=True);

        df.set_index('vendor_id', inplace=True);
        df = df.join(vendor);
        df.reset_index(names=['vendor_id'], inplace=True);

        # filter
        df = df[~df.name.str.contains('apple', False)];
        df = df[(df.updated_dt < self.REF_DT_PROMO_LUNCH) & (df.official_flag)];

        ranked_vend_prdx:pd.DataFrame = df[['prod_id', 'price', 'quantity']].groupby(['prod_id', 'price'])['quantity'].sum().reset_index();
        qtile_qty = ranked_vend_prdx.quantity.quantile(.5)
        qtile_price = ranked_vend_prdx.price.quantile(.5)
        ranked_vend_prdx = ranked_vend_prdx[(ranked_vend_prdx.quantity.ge(qtile_qty)) & (ranked_vend_prdx.quantity.ge(qtile_price))];

        df = df[df.prod_id.isin(ranked_vend_prdx['prod_id']) & (df.price > 200)];
        df = df[(df.name.str.contains('samsung smartphone', False)) | (df.vendor_id == 'VEND0016')];
        df.drop_duplicates(subset=['vendor_id','prod_id', 'price'], ignore_index=True, inplace=True);

        self.promo_src = df[['vendor_id', 'prod_id', 'price', 'name', 'biz_alias']].loc[:];
        pass;

    def get_trx_header_src(self) -> pd.DataFrame: return self.trx_header_df.loc[:];
    def get_trx_item_src(self) -> pd.DataFrame: return self.trx_item_df.loc[:];

    def get_promo_src(self): return self.promo_src.loc[:];

    def extract_promo(self) -> pd.DataFrame:
        if self.promo_df is None:

            cols = ['vendor_id', 'label', 'discount_amnt', 'lunch_schedule_dt', 'expiration_dt'];
            promo = [('VEND0016', 'Welch voucher', 50, pd.Timestamp('2023-03-15 00:00:01'), pd.Timestamp('2023-04-15 23:59:59')), #prd_id: 1002179
                    # ('Samsung offer', 45, pd.Timestamp('2023-03-15 00:00:01'), pd.Timestamp('2023-04-30 23:59:59')) #prd_id: 1002239, 1002241, 1001731, 1001793
                    ('VEND0002', 'SK offer', 45, pd.Timestamp('2023-03-15 00:00:01'), pd.Timestamp('2023-04-30 23:59:59')), #prd_id: 1002239, 1002241, 1001731, 1001793
                    ('VEND0013', 'Carter Discount', 40, pd.Timestamp('2023-03-15 00:00:01'), pd.Timestamp('2023-04-30 23:59:59'))
            ];

            df:pd.DataFrame = pd.DataFrame( data=promo, columns=cols );


            np.random.seed(self.RAND_SEED);
            deltas = np.random.randint(0, 604800, len(df.index));
            df['delta'] = deltas;
            df['delta'] = df['delta'].apply(lambda x: pd.Timedelta(x,unit='sec'));
            df['created_dt'] = self.REF_DT_PROMO_LUNCH - df['delta'];
            df['updated_dt'] = df['created_dt'];

            df['quota'] = np.random.randint(1,10,len(df.index));
            df['quota'] = df['quota'] * 100;

            df.sort_values(by=['created_dt'], ignore_index=True, inplace=True);

            init_id = np.random.randint(50216)+50216;
            df['id'] = init_id+df.index;

            # self.promo_df = df[['id', 'vendor_id', 'label', 'discount_amnt', 'lunch_schedule_dt', 'expiration_dt', 'created_dt', 'updated_dt']];
            self.promo_df = df[['id', 'vendor_id', 'label', 'discount_amnt', 'quota', 'lunch_schedule_dt', 'expiration_dt', 'created_dt', 'updated_dt']];
        return self.promo_df.loc[:];

    def extract_prod_promo(self) -> pd.DataFrame:
        if self.prod_promo_df is None:

            promo:pd.DataFrame = self.extract_promo();
            promo = promo[['id', 'label', 'created_dt']];
            # promo.set_index('id', inplace=True);

            src:pd.DataFrame = self.get_promo_src();

            # Welch's promoted product
            welch_df:pd.DataFrame = src[src.vendor_id=='VEND0016'][['prod_id']].loc[:];
            promo_indx = promo[promo.label.str.contains('Welch voucher', False)].last_valid_index();
            welch_df['promo_id'] = promo[promo.index==promo_indx]['id'].iat[-1]
            welch_df['included_flag'] = True;
            welch_df['deactivated_flag'] = False;
            welch_df['created_dt'] = promo[promo.index==promo_indx]['created_dt'].iat[-1]

            # # Samsung's promo
            # samsung_df:pd.DataFrame = src[src.vendor_id!='VEND0016'][['prod_id']].loc[:];
            # promo_indx = promo[promo.label.str.contains('SK offer', False)].last_valid_index();
            # samsung_df['promo_id'] = promo[promo.index==promo_indx]['id'].iat[-1]
            # samsung_df['included_flag'] = True;
            # samsung_df['deactivated_flag'] = False;
            # samsung_df['created_dt'] = promo[promo.index==promo_indx]['created_dt'].iat[-1]

            # SK's offer
            sk_df:pd.DataFrame = src[src.vendor_id=='VEND0002'][['prod_id']].loc[:];
            promo_indx = promo[promo.label.str.contains('SK offer', False)].last_valid_index();
            sk_df['promo_id'] = promo[promo.index==promo_indx]['id'].iat[-1]
            sk_df['included_flag'] = True;
            sk_df['deactivated_flag'] = False;
            sk_df['created_dt'] = promo[promo.index==promo_indx]['created_dt'].iat[-1]

            # Carter's discount
            carter_df:pd.DataFrame = src[src.vendor_id=='VEND0013'][['prod_id']].loc[:];
            promo_indx = promo[promo.label.str.contains('Carter Discount', False)].last_valid_index();
            carter_df['promo_id'] = promo[promo.index==promo_indx]['id'].iat[-1]
            carter_df['included_flag'] = True;
            carter_df['deactivated_flag'] = False;
            carter_df['created_dt'] = promo[promo.index==promo_indx]['created_dt'].iat[-1]

            np.random.seed(self.RAND_SEED);
            # df:pd.DataFrame = pd.concat([welch_df, samsung_df]);
            df:pd.DataFrame = pd.concat([welch_df, sk_df, carter_df]);
            df['delta'] = np.random.randint(900, 7200, len(df.index));
            df['delta'] = df['delta'].apply(lambda x: pd.Timedelta(x,unit='sec'));
            df['created_dt'] = df['created_dt'] + df['delta'];
            df['updated_dt'] = df['created_dt'];
            df.sort_values(by=['created_dt'], ignore_index=True, inplace=True);

            init_id = np.random.randint(76801)%7681;
            df['id'] = init_id + df.index;
            self.prod_promo_df = df[['id', 'prod_id', 'promo_id', 'included_flag', 'deactivated_flag', 'created_dt', 'updated_dt']];
        return self.prod_promo_df.loc[:];

    def init_promo_redeem(self) -> pd.DataFrame:
        if self.promo_redeem_df is None:
            promo:pd.DataFrame = self.extract_promo();
            np.random.seed(self.RAND_SEED);
            promo = promo[['id', 'quota', 'created_dt', 'expiration_dt']];
            # promo['len'] = np.random.randint(1,10,len(promo.index));
            # promo['len'] = promo['len']*100;
            promo['len'] = promo['quota'];

            redeem:pd.DataFrame = None;
            red_indx_offset = np.random.randint(14639);

            for i in promo.index:

                prom_index = promo['id'].iat[i];
                prom_created_dt = promo['created_dt'].iat[i];
                prom_expiration_dt = promo['expiration_dt'].iat[i];
                prom_length = promo['len'].iat[i];

                red_prom_init_idx = 0 if redeem is None else len(redeem.index);
                red_prom_init_idx += red_indx_offset;

                dat = [(red_prom_init_idx, prom_index, prom_created_dt, prom_expiration_dt, None)]*prom_length;
                cols = ['id', 'promo_id', 'created_dt', 'expiration_dt', 'activated_dt'];
                tmp:pd.DataFrame = pd.DataFrame(dat, columns=cols);
                tmp['id'] = tmp['id']+tmp.index;
                tmp['code'] = pd.util.hash_pandas_object(tmp[['id', 'promo_id']],index=False);
                tmp['code'] = tmp['code'].astype('str');
                tmp['code'] = tmp['code'].str.encode('utf-8');
                tmp['code'] = tmp['code'].apply(lambda x: hashlib.md5(x).hexdigest()[:8]);
                if redeem is None: redeem = tmp.loc[:];
                else: redeem = pd.concat([redeem, tmp]);

            self.promo_redeem_df = redeem[['id', 'promo_id', 'code', 'created_dt', 'expiration_dt', 'activated_dt']];
        return self.promo_redeem_df.loc[:];

    def get_promo_redeem(self) -> pd.DataFrame: return self.promo_redeem_df.loc[:];

    def extract_trx_promo(self) -> pd.DataFrame:
        if self.trx_promo_df is None:
            df: pd.DataFrame = None;
            txitems:pd.DataFrame = self.get_trx_item_src();
            txheader = self.get_trx_header_src()[['id', 'timestamp']];
            txheader.set_index('id', inplace=True);
            txitems.set_index('trx_header_id', inplace=True);
            txitems = txitems.join(txheader);
            txitems.reset_index(names='trx_header_id', inplace=True);

            prdx_promo:pd.DataFrame = self.extract_prod_promo()[['prod_id', 'promo_id']];
            txitems = txitems[txitems.prod_id.isin(prdx_promo.prod_id)];
            txitems.set_index('prod_id', inplace=True);
            prdx_promo.set_index('prod_id', inplace=True);
            txitems = txitems.join(prdx_promo);
            txitems.reset_index(names=['prod_id'], inplace=True);

            promo:pd.DataFrame = self.extract_promo()[['id', 'discount_amnt', 'lunch_schedule_dt', 'expiration_dt']];
            promo.set_index('id', inplace=True);
            txitems.set_index('promo_id', inplace=True);
            txitems = txitems.join(promo);
            txitems.reset_index(names=['promo_id'], inplace=True);
            txitems = txitems[txitems.timestamp.between(txitems.lunch_schedule_dt, txitems.expiration_dt)];
            txitems.drop_duplicates(subset=['trx_header_id', 'promo_id'], inplace=True);

            redcode:pd.DataFrame = self.init_promo_redeem();

            np.random.seed(self.RAND_SEED);
            txitems = txitems[['trx_header_id', 'promo_id', 'discount_amnt']]; ###rm exp_dt
            txitems['red_prob'] = np.random.normal(0,1, len(txitems.index));
            txitems = txitems[txitems.red_prob.between(-1,1)];

            for i in txitems.promo_id.drop_duplicates():
                tmp:pd.DataFrame = txitems[txitems.promo_id == i].loc[:];
                tmp.sort_values(by=['trx_header_id'], inplace=True, ignore_index=True);
                tmp['seq'] = tmp.index;

                red:pd.DataFrame = redcode[redcode.promo_id == i].loc[:];
                red.reset_index(drop=True, inplace=True);

                quota_size = len(red.index);
                spend_size = len(tmp.index);
                rand_size = spend_size if spend_size < quota_size else quota_size;

                codex_ls = np.random.choice(quota_size, rand_size, replace=False);
                codex:pd.DataFrame = pd.DataFrame(codex_ls, columns=['codex']);
                codex['seq'] = codex.index;
                codex.set_index('seq', inplace=True);
                tmp.set_index('seq', inplace=True);
                tmp = tmp.join(codex);
                tmp.reset_index(inplace=True, drop=True);
                tmp.dropna(subset=['codex'], inplace=True);
                tmp['codex'] = tmp['codex'].astype(int)
                tmp['promo_redeem_id'] = tmp['codex'].apply(lambda x: red['id'].iat[x]);

                if df is None: df = tmp.loc[:];
                else: df = pd.concat([df, tmp]);

            df.sort_values(by='trx_header_id', ignore_index=True, inplace=True);
            df['id'] = np.random.randint(290261);
            df['id'] = df['id']+ df.index;
            self.trx_promo_df = df[['id', 'trx_header_id', 'promo_id', 'promo_redeem_id', 'discount_amnt']]; ###rm exp_dt

        return self.trx_promo_df.loc[:];

    def extract_final_promo_redeem(self) -> pd.DataFrame:
        if self.final_promo_redeem_df is None:
            trx_promo:pd.DataFrame = self.extract_trx_promo();
            trx_header:pd.DataFrame = self.get_trx_header_src();
            trx_header = trx_header[['id', 'timestamp']];
            trx_header.set_index('id', inplace=True);
            trx_promo.set_index('trx_header_id', inplace=True);
            trx_promo = trx_promo.join(trx_header);
            trx_promo.reset_index(names=['trx_header_id'], inplace=True);
            trx_promo = trx_promo[['promo_redeem_id', 'timestamp']];
            trx_promo.drop_duplicates(inplace=True);

            trx_promo.set_index('promo_redeem_id', inplace=True);
            red_prom = self.get_promo_redeem();
            red_prom.set_index('id', inplace=True);
            red_prom = red_prom.join(trx_promo);
            red_prom['activated_dt'] = red_prom['timestamp'];
            red_prom.drop(columns=['timestamp'], inplace=True);
            red_prom.reset_index(names=['id'], inplace=True);
            self.final_promo_redeem_df = red_prom;
        return self.final_promo_redeem_df;

    def extract_final_trx_header(self) -> pd.DataFrame:
        if self.final_trx_header_df is None:
            trx_promo:pd.DataFrame = self.extract_trx_promo();
            trx_promo_gp = trx_promo.groupby('trx_header_id')['discount_amnt'].sum();
            trx_header:pd.DataFrame = self.get_trx_header_src();
            trx_header.set_index('id', inplace=True);
            trx_header = trx_header.join(trx_promo_gp);
            trx_header['total_discount'] = trx_header['discount_amnt'].fillna(trx_header['total_discount']);
            trx_header.reset_index(names='id', inplace=True);
            # trx_header.drop(columns=['discount_amnt'],inplace=True)
            trx_header['total_amnt'] = trx_header['sales_amnt'] - trx_header['total_discount'];
            self.final_trx_header_df = trx_header[['id', 'cust_id', 'cart_id', 'sales_amnt', 'total_discount', 'total_amnt', 'timestamp']];
        return self.final_trx_header_df.loc[:];

# vend = VendorDBGenerator().extract_vendor();
# cust = CustDBGenerator().extract_customer();
# prdx_tx = ProdxTxDBGenerator();
# prdx_tx.set_srcs(vendor=vend, customer=cust);
# prod = prdx_tx.extract_prodx();
# trx = TrxDbGenerator(prdx_tx.extract_trx());
# trx_item = trx.extract_trx_item();
# trx_header = trx.extract_trx_header();
# promo = PromoDBGenerator(vend, prod, trx_item, trx_header);
# promo.extract_final_trx_header()

#DataExtractionService

In [ ]:
class DataExtractionService:
    PRJ_PREFIX = 'mockup_ecomm_';

    vend: pd.DataFrame = None;
    cust: pd.DataFrame = None;

    prod: pd.DataFrame = None;
    prod_cat: pd.DataFrame = None;

    trx_header: pd.DataFrame = None;
    trx_item: pd.DataFrame = None;
    trx_promo: pd.DataFrame = None;

    promo:pd.DataFrame = None;
    prod_promo:pd.DataFrame = None;
    promo_redeem:pd.DataFrame = None;

    def __init__(self):
        self.vend = VendorDBGenerator().extract_vendor();
        self.cust = CustDBGenerator().extract_customer();

        prdx_tx:ProdxTxDBGenerator = ProdxTxDBGenerator();
        prdx_tx.set_srcs(vendor=self.vend, customer=self.cust);
        self.prod = prdx_tx.extract_prodx();
        self.prod_cat = prdx_tx.extract_prod_cat();

        trx:TrxDbGenerator = TrxDbGenerator(prdx_tx.extract_trx());
        # self.trx_header = trx.extract_trx_header();
        self.trx_item = trx.extract_trx_item();
        trx_header:pd.DataFrame = trx.extract_trx_header();
        # trx_item:pd.DataFrame = trx.extract_trx_item();

        promo_db:PromoDBGenerator = PromoDBGenerator(self.vend, self.prod, self.trx_item, trx_header);
        self.trx_header = promo_db.extract_final_trx_header();
        self.trx_promo = promo_db.extract_trx_promo();
        self.promo = promo_db.extract_promo();
        self.prod_promo = promo_db.extract_prod_promo();
        self.promo_redeem = promo_db.extract_final_promo_redeem();


    def extract_vend(self): return self.vend.loc[:];
    def extract_cust(self): return self.cust.loc[:];
    def extract_prod(self): return self.prod.loc[:];
    def extract_prod_cat(self): return self.prod_cat.loc[:];
    def extract_trx_header(self): return self.trx_header.loc[:];
    def extract_trx_item(self): return self.trx_item.loc[:];

    def extract_trx_promo(self): return self.trx_promo.loc[:];
    def extract_promo(self): return self.promo.loc[:];
    def extract_prod_promo(self): return self.prod_promo.loc[:];
    def extract_promo_redeem(self): return self.promo_redeem.loc[:];

    def extract_to_csv(self):
        self.vend.to_csv(self.PRJ_PREFIX + 'vend.csv', index=False);
        self.cust.to_csv(self.PRJ_PREFIX + 'cust.csv', index=False);
        self.prod.to_csv(self.PRJ_PREFIX + 'prod.csv', index=False);
        self.prod_cat.to_csv(self.PRJ_PREFIX + 'prod_cat.csv', index=False);
        self.trx_header.to_csv(self.PRJ_PREFIX + 'trx_header.csv', index=False);
        self.trx_item.to_csv(self.PRJ_PREFIX + 'trx_item.csv', index=False);
        self.trx_promo.to_csv(self.PRJ_PREFIX + 'trx_promo.csv', index=False);
        self.promo.to_csv(self.PRJ_PREFIX + 'promo.csv', index=False);
        self.prod_promo.to_csv(self.PRJ_PREFIX + 'prod_promo.csv', index=False);
        self.promo_redeem.to_csv(self.PRJ_PREFIX + 'promo_redeem.csv', index=False);
        pass;

    def download_csv(self):
        from google.colab import files;
        files.download(self.PRJ_PREFIX + 'vend.csv')
        files.download(self.PRJ_PREFIX + 'cust.csv')
        files.download(self.PRJ_PREFIX + 'prod.csv')
        files.download(self.PRJ_PREFIX + 'prod_cat.csv')
        files.download(self.PRJ_PREFIX + 'trx_header.csv')
        files.download(self.PRJ_PREFIX + 'trx_item.csv')
        files.download(self.PRJ_PREFIX + 'trx_promo.csv')
        files.download(self.PRJ_PREFIX + 'promo.csv')
        files.download(self.PRJ_PREFIX + 'prod_promo.csv')
        files.download(self.PRJ_PREFIX + 'promo_redeem.csv')

#Test

In [ ]:
serv:DataExtractionService = DataExtractionService();
serv.extract_to_csv();

In [ ]:
# serv.download_csv();

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>